In [1]:
# env: tensorflow2

import pandas as pd
import numpy as np
import os
# Keep using Keras 2
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import ydf
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.inspection import partial_dependence, permutation_importance
# from sklearn.metrics import r2_score
# from sklearn.preprocessing import OneHotEncoder
# from scipy import stats
# import matplotlib.pyplot as plt
# from statsmodels.nonparametric.smoothers_lowess import lowess
# import seaborn as sns
# from tqdm import tqdm

In [2]:

data = pd.read_csv('data.csv', index_col="Coden")
data.head()

/var/folders/11/31lkrfvs65ddtbwvncf1q57m0000gn/T/ipykernel_68774/627059569.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv', index_col="Coden")


,site.id,Crop.type,Treatment,Year.of.collection,Slope,Irrigation,Cropping.system.in.the.site,Rotational.systems,Name.of.previous.crop,Growing.days,...,Tmin10,PET10,PRE10,AI,USDA soil classification,Predicted.YGSOC,YGSOC.normalized,Yield.normalized.treatment,Yield.normalized,Yield
Coden,,,,,,,,,,,,,,,,,,,,,
1,1,Maize,N2P2K2,2010,2.0,No,2 times a year,Maize-wheat,Wheat,119,...,3.51,951.74,936.01,0.983475,NaN,14.92,0.0449,1.05,1.22,9005
2,1,Maize,N3P2K2,2010,2.0,No,2 times a year,Maize-wheat,Wheat,119,...,3.51,951.74,936.01,0.983475,NaN,14.79,0.0445,1.17,1.34,9839
3,1,Maize,N1P2K2,2010,2.0,No,2 times a year,Maize-wheat,Wheat,119,...,3.51,951.74,936.01,0.983475,NaN,22.55,0.0679,1.22,1.28,9394
4,1,Maize,N0P2K2,2010,2.0,No,2 times a year,Maize-wheat,Wheat,119,...,3.51,951.74,936.01,0.983475,NaN,19.12,0.0576,1.16,1.04,7671
5,1,Maize,Control,2010,2.0,No,2 times a year,Maize-wheat,Wheat,119,...,3.51,951.74,936.01,0.983475,NaN,30.56,0.0920,1.16,0.88,6503


In [3]:
# 数据分割函数
def prepare_data(df, crop_type: str, target_col='Yield'):
    if crop_type != 'threecrops':
        filtered = df[df['Crop.type'] == crop_type]
        cols = {
            'Maize': ["Yield", "Slope", "Irrigation", "Cropping.system.in.the.site", 
                     "Rotational.systems", "Name.of.previous.crop", "Growing.days", 
                     "GDD", "Tmax", "Tmin", "PRE", "RAD", "Soil.type", "SOC", "OP", 
                     "AK", "PH", "PK.fert", "N.fert", "SAND", "SILT", "CLAY", 
                     "AWC", "Bulk.density", "Crop.variety", "AI", "PET10"],
            'Wheat': ["Yield", "Slope", "Irrigation", "Cropping.system.in.the.site",
                     "Rotational.systems", "Name.of.previous.crop", "Growing.days",
                     "GDD", "Tmax", "Tmin", "PRE", "RAD", "Soil.type", "SOC", "OP",
                     "AK", "PH", "PK.fert", "N.fert", "SAND", "SILT", "CLAY",
                     "AWC", "Bulk.density", "Crop.variety", "AI", "PET10"],
            'Rice': ["Yield", "Slope", "Cropping.system.in.the.site", 
                    "Rotational.systems", "Growing.days", "GDD", "Tmax", "Tmin", 
                    "PRE", "RAD", "Soil.type", "SOC", "OP", "AK", "PH", "PK.fert", 
                    "N.fert", "SAND", "SILT", "CLAY", "AWC", "Bulk.density", 
                    "Crop.variety", "AI", "PET10"]
        }
        return filtered[cols[crop_type]]
    else:
        cols = ["Yield.normalized", "Slope", "Irrigation", "Cropping.system.in.the.site",
               "Rotational.systems", "Name.of.previous.crop", "Growing.days", "GDD",
               "Tmax", "Tmin", "PRE", "RAD", "Soil.type", "SOC", "OP", "AK", "PH",
               "PK.fert", "N.fert", "SAND", "SILT", "CLAY", "AWC", "Bulk.density",
               "Crop.variety", "AI", "PET10"]
        return df[cols]

# 准备数据
Maize = prepare_data(data, 'Maize')
Wheat = prepare_data(data, 'Wheat')
Rice = prepare_data(data, 'Rice')
threecrops = prepare_data(data, 'threecrops')


In [4]:
Maize.head(3)

,Yield,Slope,Irrigation,Cropping.system.in.the.site,Rotational.systems,Name.of.previous.crop,Growing.days,GDD,Tmax,Tmin,...,PK.fert,N.fert,SAND,SILT,CLAY,AWC,Bulk.density,Crop.variety,AI,PET10
Coden,,,,,,,,,,,,,,,,,,,,,
1,9005,2.0,No,2 times a year,Maize-wheat,Wheat,119,475.0,19.43,10.56,...,1,1.0,34,41,25,0.147,1.36,Medium-maturing variety,0.983475,951.74
2,9839,2.0,No,2 times a year,Maize-wheat,Wheat,119,475.0,19.43,10.56,...,1,1.5,34,41,25,0.147,1.36,Medium-maturing variety,0.983475,951.74
3,9394,2.0,No,2 times a year,Maize-wheat,Wheat,119,475.0,19.43,10.56,...,1,0.5,34,41,25,0.147,1.36,Medium-maturing variety,0.983475,951.74


In [5]:
Maize['Yield'] = Maize['Yield'] / 1000
Maize['Yield']

Coden
1        9.005
2        9.839
3        9.394
4        7.671
5        6.503
         ...  
24103    3.869
24104    6.354
24105    6.708
24106    4.743
24107    7.823
Name: Yield, Length: 24107, dtype: float64

In [6]:
# Split the dataset into a training and a testing dataset.

all_ds = Maize.sample(frac=1)
split_idx = len(all_ds) * 7 // 10
train_ds_pd = all_ds.iloc[:split_idx]
test_ds_pd = all_ds.iloc[split_idx:]

# Print the first 5 training examples
train_ds_pd.head(5)

,Yield,Slope,Irrigation,Cropping.system.in.the.site,Rotational.systems,Name.of.previous.crop,Growing.days,GDD,Tmax,Tmin,...,PK.fert,N.fert,SAND,SILT,CLAY,AWC,Bulk.density,Crop.variety,AI,PET10
Coden,,,,,,,,,,,,,,,,,,,,,
2393,11.264,20.0,Yes,1 time a year,Maize,Maize,155,1430.0,24.98,13.71,...,1,1.0,37,44,19,0.147,1.41,Medium-maturing variety,0.505863,903.07
19934,6.494,0.0,Yes,2 times a year,Maize-wheat,Wheat,112,1636.4,29.44,20.51,...,1,1.0,36,43,21,0.147,1.39,Medium-maturing variety,0.426480,1057.59
16838,7.433,0.0,Yes,2 times a year,Maize-wheat-soybean,Wheat,112,1681.8,30.65,20.07,...,0,0.0,34,48,18,0.147,1.41,Medium-maturing variety,0.439464,1074.01
2920,7.125,5.0,No,1 time a year,Maize,Maize,149,1329.9,24.72,13.26,...,0,0.0,39,40,21,0.147,1.40,Medium-maturing variety,0.695650,866.57
8755,9.068,11.0,Yes,1 time a year,Soybean,Rice,133,1217.6,24.12,14.48,...,0,0.0,28,47,25,0.147,1.35,Medium-maturing variety,0.833364,704.15


In [7]:
# Specify the model.
label = 'Yield'

# 不使用验证集，全部数据用来训练
rf_model = ydf.RandomForestLearner(label=label,
                                   task=ydf.Task.REGRESSION,
                                   num_trees=500,
                                   
                                ).train(train_ds_pd)

Train model on 16874 examples
Model trained in 0:00:03.268073


In [8]:
rf_model.describe()

In [9]:
rf_model.predict(test_ds_pd)

array([ 9.936752 ,  9.473181 ,  6.2898393, ..., 10.013467 ,  6.527079 ,
       11.856214 ], dtype=float32)

In [10]:
evaluation = rf_model.evaluate(test_ds_pd)

evaluation

Evaluation()

In [14]:
evaluation.rmse

1.2817668856651663

In [11]:
rf_model.analyze(test_ds_pd, sampling=0.1)

In [12]:
rf_model.benchmark(Maize)

Single-thread inference time per example: 69.175 us (microseconds)
Details: 72321 predictions in 0.000 seconds

Multi-thread inference time per example: 10.635 us (microseconds)
Details: 289284 predictions in 0.000 seconds using 10 threads

* Measured with the C++ serving API. See model.to_cpp().

In [13]:
print(rf_model)

Model: RANDOM_FOREST
Task: REGRESSION
Class: ydf.RandomForestModel
Use `model.describe()` for more details

